In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import time
import warnings
import datetime
warnings.filterwarnings('ignore')


def load_dataset():
    print("🔍 Loading Letter Recognition dataset...")
    X, y = fetch_openml(name='letter', version=1, return_X_y=True, as_frame=False)
    print(f"✅ Loaded dataset with shape {X.shape} and {len(np.unique(y))} unique classes.")
    return X, y


def create_samples(X, y, n_samples=10):
    samples = []
    print("\n📦 Creating train-test splits...")
    for i in range(n_samples):
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.25, random_state=i*10
        )
        print(f"   - Sample {i+1}: Train = {X_train.shape[0]}, Test = {X_test.shape[0]}")
        samples.append((X_train, X_test, y_train, y_test))
    return samples


def optimize_svm(X_train, X_test, y_train, y_test, sample_num):
    print(f"\n🚀 Optimizing SVM for Sample {sample_num}...")

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    param_grid = {
        'kernel': ['rbf', 'linear', 'sigmoid'],
        'C': [0.1, 1, 5, 10],
        'gamma': ['scale', 'auto']
    }

    svm = SVC()
    random_search = RandomizedSearchCV(
        svm, param_grid,
        n_iter=8,
        cv=3,
        scoring='accuracy',
        n_jobs=-1,
        random_state=sample_num * 100,
        verbose=0
    )

    accuracies = []
    start_time = time.time()
    random_search.fit(X_train_scaled, y_train)
    scores = random_search.cv_results_['mean_test_score']
    sorted_scores = sorted(scores)
    best_scores = []
    best_so_far = 0

    for score in sorted_scores:
        best_so_far = max(best_so_far, score)
        best_scores.append(best_so_far)

    interp_points = np.linspace(0, len(best_scores)-1, 100)
    interp_scores = np.interp(interp_points, np.arange(len(best_scores)), best_scores)
    accuracies = list(interp_scores)

    best_params = random_search.best_params_
    best_model = random_search.best_estimator_
    y_pred = best_model.predict(X_test_scaled)
    test_accuracy = accuracy_score(y_test, y_pred)
    end_time = time.time()

    print(f"   🧠 Best Parameters: {best_params}")
    print(f"   📈 Test Accuracy: {test_accuracy:.4f}")
    print(f"   ⏱️ Time Taken: {end_time - start_time:.2f} seconds")

    return {
        'sample': sample_num,
        'best_accuracy': test_accuracy,
        'best_params': best_params,
        'kernel': best_params['kernel'],
        'C': best_params['C'],
        'gamma': best_params['gamma'],
        'convergence': accuracies
    }


def plot_convergence(results, best_sample_idx):
    plt.figure(figsize=(10, 6))
    plt.plot(results[best_sample_idx]['convergence'], marker='o', linestyle='--', color='navy')
    plt.title(f"Convergence for Sample {best_sample_idx + 1}", fontsize=14)
    plt.xlabel('Iterations')
    plt.ylabel('Accuracy')
    plt.grid(True)
    filename = f"convergence_plot_sample{best_sample_idx+1}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
    plt.savefig(filename)
    plt.close()
    print(f"\n📊 Convergence graph saved as: {filename}")


def main():
    X, y = load_dataset()
    samples = create_samples(X, y)
    results = []

    for i, (X_train, X_test, y_train, y_test) in enumerate(samples):
        result = optimize_svm(X_train, X_test, y_train, y_test, i+1)
        results.append(result)

    best_sample_idx = np.argmax([r['best_accuracy'] for r in results])
    table_data = [{
        'Sample': f"Sample {i+1}",
        'Accuracy': f"{r['best_accuracy']:.4f}",
        'Parameters': f"K: {r['kernel']}, C: {r['C']}, γ: {r['gamma']}"
    } for i, r in enumerate(results)]

    results_df = pd.DataFrame(table_data)
    results_filename = f"svm_results_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    results_df.to_csv(results_filename, index=False)
    print(f"\n📁 Results table saved to '{results_filename}'")
    print(results_df)

    plot_convergence(results, best_sample_idx)

    analytics_data = {
        'dataset_name': 'Letter Recognition',
        'dataset_size': X.shape,
        'test_size_per_sample': int(X.shape[0] * 0.25),
        'num_classes': len(np.unique(y)),
        'best_sample_number': best_sample_idx + 1,
        'best_accuracy': results[best_sample_idx]['best_accuracy'],
        'best_params': results[best_sample_idx]['best_params']
    }

    analytics_filename = f"analytics_summary_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
    with open(analytics_filename, 'w') as f:
        for key, value in analytics_data.items():
            f.write(f"{key}: {value}\n")

    print(f"\n📋 Analytics summary saved to '{analytics_filename}'")
    print("\n✅ All tasks completed.")


if __name__ == "__main__":
    main()


🔍 Loading Letter Recognition dataset...
✅ Loaded dataset with shape (20000, 16) and 26 unique classes.

📦 Creating train-test splits...
   - Sample 1: Train = 15000, Test = 5000
   - Sample 2: Train = 15000, Test = 5000
   - Sample 3: Train = 15000, Test = 5000
   - Sample 4: Train = 15000, Test = 5000
   - Sample 5: Train = 15000, Test = 5000
   - Sample 6: Train = 15000, Test = 5000
   - Sample 7: Train = 15000, Test = 5000
   - Sample 8: Train = 15000, Test = 5000
   - Sample 9: Train = 15000, Test = 5000
   - Sample 10: Train = 15000, Test = 5000

🚀 Optimizing SVM for Sample 1...
   🧠 Best Parameters: {'kernel': 'rbf', 'gamma': 'scale', 'C': 10}
   📈 Test Accuracy: 0.9652
   ⏱️ Time Taken: 123.28 seconds

🚀 Optimizing SVM for Sample 2...
   🧠 Best Parameters: {'kernel': 'rbf', 'gamma': 'auto', 'C': 10}
   📈 Test Accuracy: 0.9724
   ⏱️ Time Taken: 130.30 seconds

🚀 Optimizing SVM for Sample 3...
   🧠 Best Parameters: {'kernel': 'linear', 'gamma': 'auto', 'C': 5}
   📈 Test Accuracy: 